In [1]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn 
from torchsummary import summary

from repos.training.data.preprocess import split_data
from repos.training.models.unet3D import BasicUnet3D, UnetRes3D_v1, UnetRes3D_v2
from repos.training.models.unet3D import DyUnetRes3D
from repos.training.data.trainer import Trainer
from repos.loss import Dice_BCE, FocalDiceLoss, DiceLoss
from repos import utils 
from repos.metric import Dice_3D
import parameter as para
from parameter import * 


In [3]:
import torch.nn as nn 

def segment(pretrained_path, freeze_feature, num_classes=3, act='sigmoid'):

  # backbone = DyUnetRes3D('resnet50', num_classes=num_classes)
  # backbone = UnetRes3D_v1(in_channels=1, num_classes)
  backbone = UnetRes3D_v2(in_channels=1, num_classes=num_classes)
  
  if freeze_feature:
    print('freeze_feature')
    child = nn.Sequential(*list(backbone.children())[:-1])
    for param in child.parameters():
      param.requires_grad = False  

  model = nn.Sequential()
  model.backbone = backbone

  if act == 'sigmoid':
    model.act = nn.Sigmoid()
  elif act == 'softmax':
    model.act = nn.Softmax(dim=1)

  if pretrained_path is not None:
    cp = torch.load(pretrained_path)
    model.load_state_dict(cp['model_state_dict'])
    print(pretrained_path)
    print(sum(p.numel() for p in model.parameters()))

  return model

model = segment(pretrained_path=None, freeze_feature=False, act='softmax')


In [4]:
x = torch.rand(1,1,80,160,160)
print(model(x).shape)

torch.Size([1, 3, 80, 160, 160])
